In [1]:
cd /work/FAC/FBM/DBC/cdessim2/default/dmoi/projects/aarsonline.github.io

/work/FAC/FBM/DBC/cdessim2/default/dmoi/projects/aarsonline.github.io


In [2]:
datadir = '/work/FAC/FBM/DBC/cdessim2/default/dmoi/projects/aarsonline.github.io/struct_trees/'


In [3]:
#run pdbfixer on all structs
from Bio import PDB
from pdbfixer import PDBFixer
from openmm.app import PDBFile
import glob
import sys
import tqdm
import pandas as pd
import multiprocessing as mp
#argument parser for the script 
from concurrent.futures import TimeoutError
from pebble import ProcessPool, ProcessExpired
import os
import glob

In [4]:
structsc1 = glob.glob(datadir+'c1/structs/*.pdb')
print('c1' ,  len(structsc1))
structsc2 = glob.glob(datadir+'c2/structs/*.pdb')
print('c2' ,  len(structsc2))
structs3 = glob.glob(datadir+'CRIMVLG/structs/*.pdb')
print('CRIMVLG' ,  len(structs3))

structs = structsc1 + structsc2 + structs3
print(len(structs))

c1 251
c2 298
CRIMVLG 47
596


In [5]:
#make directory for the fixed structures
os.makedirs(datadir+'c1/structs_fixed', exist_ok=True)
os.makedirs(datadir+'c2/structs_fixed', exist_ok=True)
os.makedirs(datadir+'CRIMVLG/structs_fixed', exist_ok=True)


In [8]:
def prepchain( pdbfile , chain=None, savepath=None ,unid='',  verbose = False):
    assert savepath is not None
    
    try:
        #parse the pdb file
        parser = PDB.PDBParser()
        structure = parser.get_structure(unid, pdbfile)
        if chain:
            chainID = chain
        else:
            chainID = list(structure[0].get_chains())[0].get_id()
        chain_A = structure[0][chainID]
    except:
        print("error")
        return None
    if chain_A:
        io = PDB.PDBIO()
        io.set_structure(chain_A)
        io.save(savepath)
        fixer = PDBFixer(filename=savepath)
        fixer.findNonstandardResidues()
        fixer.replaceNonstandardResidues()
        fixer.removeHeterogens(True)
        fixer.findMissingResidues()
        fixer.findMissingAtoms()
        fixer.addMissingAtoms()
        #fixer.addMissingHydrogens(7.0)
        PDBFile.writeFile(fixer.topology, fixer.positions, open(savepath, 'w'))
        return None

with ProcessPool() as pool:
    futures = [ pool.schedule( prepchain,  ( pdb , None , pdb.replace('structs', 'structs_fixed' ) , '' , False ) , timeout = 120) for pdb in structs  ]
    for future in tqdm.tqdm(futures, total=len(structs)):
        try:
            results = future.result()
        except TimeoutError as error:
            print("unstable_function took longer than %d seconds" % error.args[1])
        except ProcessExpired as error:
            print("%s. Exit code: %d" % (error, error.exitcode))
        except Exception as error:
            print("unstable_function raised %s" % error)
            print(error.traceback)  # Python's traceback of remote process
    pool.close()
    pool.join()



  0%|          | 0/596 [00:00<?, ?it/s]/work/FAC/FBM/DBC/cdessim2/default/dmoi/miniconda3/envs/torch/lib/python3.12/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 5080.
  warnings.warn(
/work/FAC/FBM/DBC/cdessim2/default/dmoi/miniconda3/envs/torch/lib/python3.12/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 5106.
  warnings.warn(
  1%|          | 5/596 [00:38<1:00:44,  6.17s/it]/work/FAC/FBM/DBC/cdessim2/default/dmoi/miniconda3/envs/torch/lib/python3.12/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 7212.
  warnings.warn(
/work/FAC/FBM/DBC/cdessim2/default/dmoi/miniconda3/envs/torch/lib/python3.12/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 7382.
  warnings.warn(
  9%|▉         | 54/596 [01:09<06:42,  1.35it/s] /work/FAC/FBM/D

unstable_function took longer than 120 seconds


 42%|████▏     | 251/596 [04:29<05:55,  1.03s/it]/work/FAC/FBM/DBC/cdessim2/default/dmoi/miniconda3/envs/torch/lib/python3.12/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 9833.
  warnings.warn(
/work/FAC/FBM/DBC/cdessim2/default/dmoi/miniconda3/envs/torch/lib/python3.12/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 9859.
  warnings.warn(
 46%|████▌     | 274/596 [04:31<03:36,  1.49it/s]/work/FAC/FBM/DBC/cdessim2/default/dmoi/miniconda3/envs/torch/lib/python3.12/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 12930.
  warnings.warn(
/work/FAC/FBM/DBC/cdessim2/default/dmoi/miniconda3/envs/torch/lib/python3.12/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 12952.
  warnings.warn(
/work/FAC/FBM/DBC/cdessim2/default/dmoi/miniconda3/e

In [6]:
import sys
import os
sys.path.append('../snake_tree/src')

import corecut
import numpy as np
import pandas as pd
import foldseek2tree

def structblob2tree(input_folder, outfolder, overwrite = False,
             fastmepath = 'fastme', quicktreepath = 'quicktree' , 
         foldseekpath = 'foldseek' , delta = 0.0001 ,
       correction = False , kernel = 'fident' , core = False 
       , hittresh = .8 , minthresh = .6 , swapids = False):
    '''
    run fold tree pipeline for a folder of pdb files
    
    Parameters
    ----------
    input_folder : str
        path to folder with pdb files   
    outfolder : str
        path to output folder   
    overwrite : bool
        overwrite existing foldseek output  
    fastmepath : str    
        path to fastme executable
    quicktreepath : str 
        path to quicktree executable
    foldseekpath : str  
        path to foldseek executable 
    delta : float   
        small number to replace negative branch lengths with, default is .0001
    correction : str    
        correction method to use, either 'tajima' or 'none'
    kernel : str    
        kernel to use, either 'fident', 'lddt' or 'alntmscore'
    
    '''
    #check if the foldseek output is already there
    if os.path.exists(outfolder + 'res.m8') and overwrite == False:
        print('found foldseek output, skipping foldseek')
        alnres = outfolder + 'res.m8'
    else:
        alnres = foldseek2tree.runFoldseek_allvall_EZsearch(input_folder , outfolder + 'res.m8', foldseekpath = foldseekpath)
    
    if core == True:

        corecut.extract_core( alnres , alnres+'.core.csv',  hitthresh = .8 ,minthresh = .6, structfolder= input_folder.split('/')[-2]+'/' , corefolder = input_folder+'core_structs/'  )
        if os.path.exists(outfolder + 'res.m8') and overwrite == False:
            print('found foldseek core output, skipping foldseek')
            alnres = outfolder + 'core.res.m8'
        else:
            alnres = foldseek2tree.runFoldseek_allvall_EZsearch(input_folder , outfolder + 'core.res.m8', foldseekpath = foldseekpath)
    
    
    res = pd.read_table(alnres , header = None )
    res[0] = res[0].map(lambda x :x.replace('.pdb', ''))
    res[1] = res[1].map(lambda x :x.replace('.pdb', ''))
    res.columns = 'query,target,fident,alnlen,mismatch,gapopen,qstart,qend,tstart,tend,evalue,bits,lddt,lddtfull,alntmscore'.split(',')
    ids = sorted(list( set(list(res['query'].unique()) + list(res['target'].unique()))))
    if swapids:
        res['numerical_query'] = res['query'].map(lambda x : ids.index(x))
        res['numerical_target'] = res['target'].map(lambda x : ids.index(x))
    
    pos = { protid : i for i,protid in enumerate(ids)}
    if swapids:
        ids = [ str(i) for i in range(len(ids))]

    matrices = { kernel:np.zeros((len(pos), len(pos)))  }
    print(res)

    #calc kernel for tm, aln score, lddt
    for idx,row in res.iterrows():
        for k in matrices:
            matrices[k][pos[row['query']] , pos[row['target']]] += row[k]
            matrices[k][pos[row['target']] , pos[row['query']]] += row[k]
    trees = {}
    for i,k in enumerate(matrices):
        matrices[k] /= 2
        matrices[k] = 1-matrices[k]
        matrices[k] = np.clip(matrices[k], 0, 1)
        
        print(matrices[k], np.amax(matrices[k]), np.amin(matrices[k]) )
        if correction:
            if kernel == 'fident':
                factor = .93
            else:
                factor = 1
            matrices[k] = foldseek2tree.Tajima_dist(matrices[k], bfactor = factor)
        np.save( input_folder + k + '_distmat.npy' , matrices[k])
        distmat_txt = foldseek2tree.distmat_to_txt( ids , matrices[k] , outfolder + k + '_distmat.txt' )
        out_tree = foldseek2tree.runFastme(  fastmepath = fastmepath , clusterfile = distmat_txt )
        out_tree = foldseek2tree.postprocess(out_tree, input_folder + 'structblob_tree.nwk' , delta = delta)
        trees[k] = out_tree
    return alnres, trees

In [7]:


os.makedirs(datadir+'c1/ft1tree', exist_ok=True)

os.makedirs(datadir+'c2/ft1tree', exist_ok=True)

os.makedirs(datadir+'CRIMVLG/ft1tree', exist_ok=True)

os.makedirs(datadir+'c1/ft1corecut', exist_ok=True)

os.makedirs(datadir+'c2/ft1corecut', exist_ok=True)

os.makedirs(datadir+'CRIMVLG/ft1corecut', exist_ok=True)


for folder in ['c1', 'c2', 'CRIMVLG']:
    #foldtree basic
    structblob2tree(input_folder = datadir+folder +'/structs_fixed/', outfolder = datadir+folder+'/ft1tree' , overwrite = False,
                fastmepath = 'fastme', quicktreepath = 'quicktree' , 
            foldseekpath = 'foldseek' , delta = 0.0001 ,
        correction = True , kernel = 'fident' , core = False 
        , hittresh = .8 , minthresh = .6  , swapids=True) 

    structblob2tree(input_folder = datadir+folder +'/structs_fixed/' , outfolder = datadir+folder+'/ft1corecut' , overwrite = False,
                fastmepath = 'fastme', quicktreepath = 'quicktree' , 
            foldseekpath = 'foldseek' , delta = 0.0001 ,
        correction = True , kernel = 'fident' , core = True 
        , hittresh = .8 , minthresh = .6 , swapids=True) 

found foldseek output, skipping foldseek
                                                   query  \
0      MetRS_AF_Arch_Candidatus_Korarchaeum_cryptofil...   
1      MetRS_AF_Arch_Candidatus_Korarchaeum_cryptofil...   
2      MetRS_AF_Arch_Candidatus_Korarchaeum_cryptofil...   
3      MetRS_AF_Arch_Candidatus_Korarchaeum_cryptofil...   
4      MetRS_AF_Arch_Candidatus_Korarchaeum_cryptofil...   
...                                                  ...   
61398  LeuRS-B_AF_Mito_Drosophila_melanogaster_gene38581   
61399  LeuRS-B_AF_Mito_Drosophila_melanogaster_gene38581   
61400  LeuRS-B_AF_Mito_Drosophila_melanogaster_gene38581   
61401  LeuRS-B_AF_Mito_Drosophila_melanogaster_gene38581   
61402  LeuRS-B_AF_Mito_Drosophila_melanogaster_gene38581   

                                                  target  fident  alnlen  \
0      MetRS_AF_Arch_Candidatus_Korarchaeum_cryptofil...   1.000     559   
1      MetRS_AF_Arch_Aciduliprofundum_boonei_T469_gen...   0.414     545   
2      Met

processed: 65:  26%|██▌       | 64/249 [00:00<00:02, 84.88it/s]

[[0.         0.03212851 0.03212851 ... 0.02409639 0.01606426 0.00401606]]                                                query  \
11633  IleRS_AF_Euk_Ustilago_maydis_521_gene23568123   
11634  IleRS_AF_Euk_Ustilago_maydis_521_gene23568123   
11635  IleRS_AF_Euk_Ustilago_maydis_521_gene23568123   
11636  IleRS_AF_Euk_Ustilago_maydis_521_gene23568123   
11637  IleRS_AF_Euk_Ustilago_maydis_521_gene23568123   
...                                              ...   
11874  IleRS_AF_Euk_Ustilago_maydis_521_gene23568123   
11875  IleRS_AF_Euk_Ustilago_maydis_521_gene23568123   
11876  IleRS_AF_Euk_Ustilago_maydis_521_gene23568123   
11877  IleRS_AF_Euk_Ustilago_maydis_521_gene23568123   
11878  IleRS_AF_Euk_Ustilago_maydis_521_gene23568123   

                                                  target  fident  alnlen  \
11633      IleRS_AF_Euk_Ustilago_maydis_521_gene23568123   1.000    1083   
11634               IleRS_AF_Mito_Homo_sapiens_gene55699   0.352    1100   
11635  IleRS_AF_Euk_Cyani

processed: 94:  37%|███▋      | 93/249 [00:01<00:01, 84.98it/s]

[[0.         0.01204819 0.01606426 ... 0.02409639 0.02008032 0.01204819]]                                                    query  \
18789  ValRS_AF_Euk_Plasmodium_knowlesi_strain_H_gene...   
18790  ValRS_AF_Euk_Plasmodium_knowlesi_strain_H_gene...   
18791  ValRS_AF_Euk_Plasmodium_knowlesi_strain_H_gene...   
18792  ValRS_AF_Euk_Plasmodium_knowlesi_strain_H_gene...   
18793  ValRS_AF_Euk_Plasmodium_knowlesi_strain_H_gene...   
...                                                  ...   
19031  ValRS_AF_Euk_Plasmodium_knowlesi_strain_H_gene...   
19032  ValRS_AF_Euk_Plasmodium_knowlesi_strain_H_gene...   
19033  ValRS_AF_Euk_Plasmodium_knowlesi_strain_H_gene...   
19034  ValRS_AF_Euk_Plasmodium_knowlesi_strain_H_gene...   
19035  ValRS_AF_Euk_Plasmodium_knowlesi_strain_H_gene...   

                                                  target  fident  alnlen  \
18789  ValRS_AF_Euk_Plasmodium_knowlesi_strain_H_gene...   1.000    1070   
18790  ValRS_AF_Euk_Trypanosoma_brucei_brucei_TREU927

...

processed: 234:  94%|█████████▍| 234/249 [00:02<00:00, 82.65it/s]

[[0.         0.10040161 0.1124498  ... 0.00803213 0.00803213 0.00803213]]                                                 query  \
53552  LeuRS-A_AF_Euk_Arabidopsis_thaliana_gene837489   
53553  LeuRS-A_AF_Euk_Arabidopsis_thaliana_gene837489   
53554  LeuRS-A_AF_Euk_Arabidopsis_thaliana_gene837489   
53555  LeuRS-A_AF_Euk_Arabidopsis_thaliana_gene837489   
53556  LeuRS-A_AF_Euk_Arabidopsis_thaliana_gene837489   
...                                               ...   
53788  LeuRS-A_AF_Euk_Arabidopsis_thaliana_gene837489   
53789  LeuRS-A_AF_Euk_Arabidopsis_thaliana_gene837489   
53790  LeuRS-A_AF_Euk_Arabidopsis_thaliana_gene837489   
53791  LeuRS-A_AF_Euk_Arabidopsis_thaliana_gene837489   
53792  LeuRS-A_AF_Euk_Arabidopsis_thaliana_gene837489   

                                                  target  fident  alnlen  \
53552     LeuRS-A_AF_Euk_Arabidopsis_thaliana_gene837489   1.000    1091   
53553  LeuRS-A_AF_Euk_Drosophila_melanogaster_gene326262   0.474    1099   
53554  LeuRS-

processed: 249: 100%|██████████| 249/249 [00:02<00:00, 84.46it/s]


[[0.         0.07630522 0.12048193 ... 0.04819277 0.04016064 0.02409639]]                                                    query  \
59452  IleRS_AF_Bact_Thermus_thermophilus_HB8_gene316...   
59453  IleRS_AF_Bact_Thermus_thermophilus_HB8_gene316...   
59454  IleRS_AF_Bact_Thermus_thermophilus_HB8_gene316...   
59455  IleRS_AF_Bact_Thermus_thermophilus_HB8_gene316...   
59456  IleRS_AF_Bact_Thermus_thermophilus_HB8_gene316...   
...                                                  ...   
59684  IleRS_AF_Bact_Thermus_thermophilus_HB8_gene316...   
59685  IleRS_AF_Bact_Thermus_thermophilus_HB8_gene316...   
59686  IleRS_AF_Bact_Thermus_thermophilus_HB8_gene316...   
59687  IleRS_AF_Bact_Thermus_thermophilus_HB8_gene316...   
59688  IleRS_AF_Bact_Thermus_thermophilus_HB8_gene316...   

                                                  target  fident  alnlen  \
59452  IleRS_AF_Bact_Thermus_thermophilus_HB8_gene316...   1.000    1043   
59453                        IleRS_PDB_Bact_T_therm_1

TypeError: 'exist_ok' is an invalid keyword argument for mkdir()

In [ ]:
import subprocess
#run ml and foldtree trees on full and core
def MLpartion(  ):
    #run snakemake to generate the partition tree
    

In [ ]:
#madroot trees
madpath = '../../snake_tree/madroot/mad'
def madroot(  madpath , treefile):
    cmd = f'{madpath} {treefile}'
    subprocess.run(cmd, shell=True)
    return treefile+'.rooted'

In [ ]:
# run CRIMVLG domain tree the pdb files
#foldtree basic
foldtree.structblob2tree(input_folder = './CRIMVLG/structs/', outfolder, overwrite = False,
             fastmepath = 'fastme', quicktreepath = 'quicktree' , 
         foldseekpath = 'foldseek' , delta = 0.0001 ,
       correction = True , kernel = 'fident' , core = False 
       , hittresh = .8 , minthresh = .6 ) 
